## Download and Format Images 

Decide below which "concepts" to include. These are the classes you want to be able to identify.

In [1]:
concepts_to_include = [
    "Asteroidea",
    "Ophiuroidea",
]

In [2]:
# Download the images from the fathomnet database
import os

data_directory = os.path.join('datasets', 'sea_data')

!fathomnet-generate -c '{', '.join(concepts_to_include)}' --format voc --img-download "{os.path.join(data_directory, 'images')}" --output "{os.path.join(data_directory, 'labels')}"

100% (2667 of 2667) |####################| Elapsed Time: 1:00:05 Time:  1:00:05


In [3]:
# Build YAML
# This is what YOLO uses to route to its data and read the txt files' labeling

!pip install pyyaml
import yaml

train_dir = 'train'
val_dir = 'train'
test_dir = 'train'

def createYamlFile(yaml_name='data'):
    data = {
        "path": 'sea_data',
        "train": [os.path.join('images', train_dir)],
        "val": [os.path.join('images', val_dir)],
        "test": [os.path.join('images', test_dir)],
        "names": {str(idx): class_name for idx, class_name in enumerate(concepts_to_include)}
    }

    with open(os.path.join(data_directory, f"{yaml_name}.yaml"), "w") as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False)

createYamlFile()

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import shutil

def renameAndMoveImgPair(uid, new_id, dest_dir=train_dir):
    curr_location = os.path.join(data_directory, 'images')
    src_file = os.path.join(curr_location, f"{uid}.png")

    file_exists = os.path.isfile(os.path.join(curr_location, f"{uid}.png"))
    if not file_exists: return

    os.makedirs(os.path.join(curr_location, dest_dir), exist_ok=True)
    dest_file = os.path.join(curr_location, dest_dir, f"{new_id}.png")

    shutil.move(src_file, dest_file)


# renameAndMoveImgPair('0a1ee4e2-f85f-45ac-8cc6-1ce9299c6444', '001', train_dir)

In [5]:
import xml.etree.ElementTree as ET
import cv2

def createTxtFile(uid, new_id, dest_dir=train_dir):
    curr_location = os.path.join(data_directory, 'labels')
    
    tree = ET.parse(os.path.join(curr_location, f'{uid}.xml'))
    root = tree.getroot()

    file_destination = os.path.join(curr_location, dest_dir)
    os.makedirs(file_destination, exist_ok=True)

    image = cv2.imread(os.path.join(data_directory, 'images', dest_dir, f"{new_id}.png"))
    img_height, img_width, _ = image.shape

    with open(os.path.join(file_destination, f'{new_id}.txt'), 'w') as txt_file:
        for obj in root.findall('.//object'):
            name_key = concepts_to_include.index(obj.find('name').text)
    
            xmin = int(obj.find('bndbox/xmin').text)
            ymin = int(obj.find('bndbox/ymin').text)
            width = int(obj.find('bndbox/xmax').text) - xmin
            height = int(obj.find('bndbox/ymax').text) - ymin
    
            line = f"{name_key} {xmin / img_width:6f} {ymin / img_height:6f} {width / img_width:6f} {height / img_height:6f}"
            txt_file.write(line + '\n')

    os.remove(os.path.join(curr_location, f'{uid}.xml'))

# createTxtFile('0a1ee4e2-f85f-45ac-8cc6-1ce9299c6444', '001', train_dir)

In [ ]:
directory_path = os.path.join(data_directory, 'images')

# Iterate through each file in the directory
for i, filename in enumerate(os.listdir(directory_path)):
    if not os.path.isfile(os.path.join(directory_path, filename)): 
        continue

    root_name, _ = os.path.splitext(filename)
    
    renameAndMoveImgPair(root_name, "{:012d}".format(i+1))
    createTxtFile(root_name, "{:012d}".format(i+1))
 

## Create Model and begin training

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml') 
model = YOLO('yolov8n.pt') 
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train the model
results = model.train(data=f"{data_directory}/data.yaml", epochs=10, imgsz=720,486)